In [ ]:
import pandas as pd
#import numpy as np

#importing plotly and cufflinks in offline mode
import plotly.plotly as py
import plotly.graph_objs as go
import cufflinks
import plotly.offline as pyo
from plotly.offline import plot, iplot, init_notebook_mode
pyo.init_notebook_mode()
cufflinks.go_offline()
cufflinks.set_config_file(world_readable=True, theme='white')

In [ ]:
#Read CBC output file
df = pd.read_csv('cbcoutput.txt', sep='\t')

In [ ]:
df.columns = ['temp']
df[['temp','value']] = df['temp'].str.split(')', expand=True)
df = df.applymap(lambda x: x.strip() if isinstance(x,str) else x)
df[['temp','parameter']] = df['temp'].str.split(' ', expand=True)
df[['parameter','id']] = df['parameter'].str.split('(', expand=True)
df['value'] = df['value'].str.replace(' 0','')
df = df.drop('temp', axis=1)
df = df[~df['value'].str.contains('e-')]

In [ ]:
params = df.parameter.unique()
all_params = {}
cols = {'NewCapacity':['r','t','y'],
            'AccumulatedNewCapacity':['r','t','y'], 
            'TotalCapacityAnnual':['r','t','y'],
            'CapitalInvestment':['r','t','y'],
            'AnnualVariableOperatingCost':['r','t','y'],
            'AnnualFixedOperatingCost':['r','t','y'],
            'SalvageValue':['r','t','y'],
            'DiscountedSalvageValue':['r','t','y'],
            'TotalTechnologyAnnualActivity':['r','t','y'],
            'RateOfActivity':['r','l','t','m','y'],
            'RateOfTotalActivity':['r','t','l','y'],
            'Demand':['r','l','f','y'],
            'TotalAnnualTechnologyActivityByMode':['r','t','m','y'],
            'TotalTechnologyModelPeriodActivity':['r','t'],
            'ProductionByTechnologyAnnual':['r','t','f','y'],
            'AnnualTechnologyEmissionByMode':['r','t','e','m','y'],
            'AnnualTechnologyEmission':['r','t','e','y'],
            'AnnualEmissions':['r','e','y']}

for each in params:
    df_p = df[df.parameter == each]
    df_p[cols[each]] = df_p['id'].str.split(',',expand=True)
    cols[each].append('value')
    df_p = df_p[cols[each]] # Reorder dataframe to include 'value' as last column
    all_params[each] = pd.DataFrame(df_p) # Create a dataframe for each parameter
    df_p = df_p.rename(columns={'value':each})
    #df_p.to_csv(str(each) + '.csv', index=None) # Print data for each paramter to a CSV file

In [ ]:
# List of columns for detailed energy tables and figures
years = pd.Series(range(2015,2031))

det_col = {'BIO':'Biomass',
           'COA':'Coal',
           'CRU':'Crude oil',
           'DSL':'Diesel',
           'ELC':'Electricity',
           'GEO':'Geothermal',
           'GSL':'Gasoline',
           'HFO':'HFO',
           'HYD':'Hydro',
           'JFL':'Jet fuel',
           'LNG':'LNG',
           'LPG':'LPG',
           'PCK':'Petroleum coke',
           'NGS':'Natural gas',
           'OHC':'Other hydrocarbons',
           'PET':'Petroleum products',
           'SOL':'Solar',
           'URN':'Nuclear',
           'WAS':'MSW',
           'WAT':'Water',
           'LND':'Land',
           'WND':'Wind',
           'CP1':'Coffee',
           'CP2':'Cotton',
           'CP3':'Peanuts',
           'CP4':'Maize',
           'CP5':'Rice',
           'CP6':'Soybean',
           'CP7':'Sugarcane',
           'CP8':'Tea',
           'AGR':'Agriculture',
           'BAR':'Barren land',
           'FOR': 'Forests',
           'GRS': 'Grassland & woodland',
           'SET':'Built-up land',
           'OTH':'Other agricultural land',
           'PWR':'Power sector',
           'PUB':'Public supply',
           'EVT':'Evapotranspiration',
           'PRC':'Precipitation',
           'GRC':'Groundwater recharge',
           'SUR':'Surface water'}

# List of columns for aggregated energy tables and figures
agg_col = {'Coal':['Coal'],
           'Oil': ['Diesel','HFO','JFL','Crude oil','Petroleum coke'],
           'Gas': ['Natural gas','LNG','LPG'],
           'Hydro': ['Hydro'],
           'Nuclear': ['Nuclear'],
           'Other renewables': ['Biomass','Geothermal','Solar','MSW','Wind']}

color_dict = {'Biomass':'forestgreen',
              'Coal':'black',
              'Crude oil':'red',
              'Diesel':'firebrick',
              'Electricity':'grey',
              'Geothermal':'purple',
              'Gasoline':'orange',
              'HFO':'brown',
              'Hydro':'blue',
              'Jet fuel':'firebrick',
              'LNG':'purple',
              'LPG':'gold',
              'Natural gas':'orange',
              'Other hydrocarbons':'grey',
              'Petroleum products':'red',
              'Petroleum coke':'red',
              'Solar':'gold',
              'Nuclear':'grey',
              'MSW':'green',
              'Wind':'violet',
              'Oil':'firebrick',
              'Gas':'orange',
              'Other renewables':'green',
              'Coffee':'brown',
              'Cotton':'grey',
              'Peanuts':'orange',
              'Maize':'gold',
              'Rice':'green',
              'Soybean':'firebrick',
              'Sugarcane':'purple',
              'Tea':'pink',
              'Agriculture':'brown',
              'Barren land':'grey',
              'Forests':'forestgreen',
              'Grassland & woodland':'darkgreen',
              'Built-up land':'black',
              'Other agricultural land':'gold',
              'Water':'blue',
              'Power sector':'red',
              'Public supply':'grey',
              'Irrigation':'brown',
              'Evapotranspiration':'green',
              'Precipitation':'blue',
              'Groundwater recharge':'brown',
              'Surface water':'red'}

In [ ]:
def df_filter(df,lb,ub,t_exclude):
    df['t'] = df['t'].str[lb:ub]
    df['value'] = df['value'].astype('float64')
    df = df[~df['t'].isin(t_exclude)].pivot_table(index='y', 
                                          columns='t',
                                          values='value', 
                                          aggfunc='sum').reset_index().fillna(0)
    df = df.reindex(sorted(df.columns), axis=1).set_index('y').reset_index().rename(columns=det_col)
    df['y'] = years
    return df

def df_plot(df,y_title,p_title):
    return df.iplot(x='y',
             kind='bar', 
             barmode='stack',
             xTitle='Year',
             yTitle=y_title,
             color=[color_dict[x] for x in df.columns if x != 'y'],
             title=p_title)

In [ ]:
# Power generation capacity (detailed)
cap_df = all_params['TotalCapacityAnnual'][all_params['TotalCapacityAnnual'].t.str.startswith('PWR')].drop('r', axis=1)
cap_df = df_filter(cap_df,3,6,['TRN'])
df_plot(cap_df,'Gigawatts (GW)','Power Generation Capacity (Detail)')

In [ ]:
# Power generation capacity (Aggregated)
cap_agg_df = pd.DataFrame(columns=agg_col)
cap_agg_df.insert(0,'y',cap_df['y'])
cap_agg_df  = cap_agg_df.fillna(0.00)

for each in agg_col:
    for tech_exists in agg_col[each]:
        if tech_exists in cap_df.columns:
            cap_agg_df[each] = cap_agg_df[each] + cap_df[tech_exists]
            cap_agg_df[each] = cap_agg_df[each].round(2)

df_plot(cap_agg_df,'Gigawatts (GW)','Power Generation Capacity (Aggregate)')

In [ ]:
#Power generation (Detailed)
gen_df = all_params['ProductionByTechnologyAnnual'][all_params['ProductionByTechnologyAnnual'].t.str.startswith('PWR') & 
                                                   all_params['ProductionByTechnologyAnnual'].f.str.startswith('ELC001')].drop('r', axis=1)
gen_df = df_filter(gen_df,3,6,['TRN'])
df_plot(gen_df,'Petajoules (PJ)','Power Generation (Detail)')

In [ ]:
# Power generation (Aggregated)
gen_agg_df = pd.DataFrame(columns=agg_col)
gen_agg_df.insert(0,'y',gen_df['y'])
gen_agg_df  = gen_agg_df.fillna(0.00)

for each in agg_col:
    for tech_exists in agg_col[each]:
        if tech_exists in gen_df.columns:
            gen_agg_df[each] = gen_agg_df[each] + gen_df[tech_exists]
            gen_agg_df[each] = gen_agg_df[each].round(2)

df_plot(gen_agg_df,'Petajoules (PJ)','Power Generation (Aggregate)')

In [ ]:
# Fuel use for power generation
gen_use_df = all_params['ProductionByTechnologyAnnual'][all_params['ProductionByTechnologyAnnual'].t.str.startswith('DEMPWR')].drop('r', axis=1)
gen_use_df = df_filter(gen_use_df,6,9,[])
df_plot(gen_use_df,'Petajoules (PJ)','Power Generation (Fuel use)')

In [ ]:
#Domestic fuel production
fuels = ['OHC', 'GSL','DSL','LPG', 'JFL','HFO','NGS']

dom_prd_df = all_params['ProductionByTechnologyAnnual'][all_params['ProductionByTechnologyAnnual'].t.str.startswith('MIN')|
                                                       all_params['ProductionByTechnologyAnnual'].t.str.startswith('RNW')].drop('r', axis=1)
dom_prd_df = df_filter(dom_prd_df,3,6,[])

for each in dom_prd_df.columns:
    if each in ['Land','Water','Geothermal','Hydro','Solar','Wind']:
        dom_prd_df = dom_prd_df.drop(each, axis=1)
df_plot(dom_prd_df,'Petajoules (PJ)','Domestic energy production')

In [ ]:
#Energy imports
ene_imp_df = all_params['ProductionByTechnologyAnnual'][all_params['ProductionByTechnologyAnnual'].t.str.startswith('IMP')].drop('r', axis=1)
ene_imp_df = df_filter(ene_imp_df,3,6,[])
#if len(ene_imp_df.columns) > 1:
#    df_plot(ene_imp_df,'Petajoules (PJ)','Energy imports')

In [ ]:
#Energy exports
ene_exp_df = all_params['TotalTechnologyAnnualActivity'][all_params['TotalTechnologyAnnualActivity'].t.str.startswith('EXP')].drop('r', axis=1)
ene_exp_df = df_filter(ene_exp_df,3,6,[])
if len(ene_exp_df.columns) > 1:
    df_plot(ene_exp_df,'Petajoules (PJ)','Energy exports')

In [ ]:
cap_cos_df = all_params['CapitalInvestment'][all_params['CapitalInvestment'].t.str.startswith('PWR')].drop('r', axis=1)
cap_cos_df = df_filter(cap_cos_df,3,6,['TRN'])
df_plot(cap_cos_df,'Million $','Capital Investment')

In [ ]:
ele_cos_df = pd.DataFrame(columns=['Total capital investment', 'Capital costs'])
ele_cos_df.insert(0,'y',years)
ele_cos_df['Total capital investment'] = cap_cos_df.iloc[:,1:].sum(axis=1)
ele_cos_df['Capital costs'] = 0
ele_cos_df = ele_cos_df.fillna(0)

R = 0.1 # Discount rate
n = 20 # Amortization period

for i in ele_cos_df['y']:
    for j in ele_cos_df['y']:
        if i < j + n and i >= j:
            ele_cos_df.loc[ele_cos_df['y']==i,'Capital costs'] = ele_cos_df.loc[ele_cos_df['y']==i,'Capital costs'] + (((ele_cos_df.loc[ele_cos_df['y']==j,'Total capital investment'].iloc[0])*R)/(1-(1+R)**(-n)))

ele_cos_df.drop('Total capital investment', axis=1, inplace=True)

In [ ]:
fix_cos_df = all_params['AnnualFixedOperatingCost'][all_params['AnnualFixedOperatingCost'].t.str.startswith('PWR')].drop('r', axis=1)
fix_cos_df = df_filter(fix_cos_df,3,6,['TRN'])

var_cos_df = all_params['AnnualVariableOperatingCost'][all_params['AnnualVariableOperatingCost'].t.str.startswith('PWR')].drop('r', axis=1)
var_cos_df = df_filter(var_cos_df,3,6,['TRN'])

dis_cos_df = all_params['AnnualVariableOperatingCost'][all_params['AnnualVariableOperatingCost'].t.str.startswith('DEMPWR')].drop('r', axis=1)
dis_cos_df = df_filter(dis_cos_df,6,9,[])

dom_val_df = all_params['AnnualVariableOperatingCost'][all_params['AnnualVariableOperatingCost'].t.str.startswith('MIN')|
                                                       all_params['AnnualVariableOperatingCost'].t.str.startswith('RNW')].drop('r', axis=1)
dom_val_df = df_filter(dom_val_df,3,6,[])
for each in dom_val_df.columns:
    if each in ['Land','Water','Geothermal','Hydro','Solar','Wind']:
        dom_val_df = dom_val_df.drop(each, axis=1)
        
imp_val_df = all_params['AnnualVariableOperatingCost'][all_params['AnnualVariableOperatingCost'].t.str.startswith('IMP')].drop('r', axis=1)
imp_val_df = df_filter(imp_val_df,3,6,[])

exp_val_df = all_params['AnnualVariableOperatingCost'][all_params['AnnualVariableOperatingCost'].t.str.startswith('EXP')].drop('r', axis=1)
exp_val_df = df_filter(exp_val_df,3,6,[])

In [ ]:
temp_col_list = []
temp_col_list = dom_val_df.columns

if len(imp_val_df.columns) > 1:
    temp_col_list = temp_col_list.append(imp_val_df.columns)

if len(exp_val_df.columns) > 1:
    temp_col_list = temp_col_list.append(exp_val_df.columns)

fue_val_df = pd.DataFrame(columns=list(set(temp_col_list)))
fue_val_df['y'] = years
fue_val_df = fue_val_df.reindex(sorted(fue_val_df.columns),axis=1).set_index('y').reset_index()

for each in dom_prd_df.columns:
    if each != 'y':
        fue_val_df[each] = dom_val_df[each]
        fue_val_df = fue_val_df.fillna(0)
for each in imp_val_df.columns :
    if each != 'y' and len(imp_val_df.columns) > 1:
        fue_val_df[each] = fue_val_df[each] + imp_val_df[each]
        fue_val_df = fue_val_df.fillna(0)
for each in exp_val_df.columns:
    if each != 'y' and len(ene_exp_df.columns) > 1:
        fue_val_df[each] = fue_val_df[each] - exp_val_df[each]

In [ ]:
temp_col_list = []
temp_col_list = dom_prd_df.columns
if len(ene_imp_df.columns) > 1:
    temp_col_list = temp_col_list.append(ene_imp_df.columns)
if len(ene_exp_df.columns) > 1:
    temp_col_list = temp_col_list.append(ene_exp_df.columns)

fue_prd_df = pd.DataFrame(columns=list(set(temp_col_list)))
fue_prd_df['y'] = years
fue_prd_df = fue_prd_df.reindex(sorted(fue_prd_df.columns),axis=1).set_index('y').reset_index()

for each in dom_prd_df.columns:
    if each != 'y':
        fue_prd_df[each] = dom_prd_df[each]
        fue_prd_df = fue_prd_df.fillna(0)
for each in ene_imp_df.columns:
    if each != 'y' and len(ene_imp_df.columns) > 1:
        fue_prd_df[each] = fue_prd_df[each] + ene_imp_df[each]
        fue_prd_df = fue_prd_df.fillna(0)
for each in ene_exp_df.columns:
    if each != 'y' and len(ene_exp_df.columns) > 1:
        fue_prd_df[each] = fue_prd_df[each] - ene_exp_df[each]

In [ ]:
fue_cos_df = pd.DataFrame(columns=list(set(temp_col_list)))
fue_cos_df['y'] = years

fue_cos_df = (fue_val_df/fue_prd_df)*gen_use_df
fue_cos_df = fue_cos_df.fillna(0)
fue_cos_df = fue_cos_df.reindex(sorted(fue_cos_df.columns),axis=1).set_index('y').reset_index()
fue_cos_df['y'] = years

In [ ]:
ele_cos_df['Electricity generation'] = gen_df.iloc[:,1:].sum(axis=1)/3.6
ele_cos_df['Capital costs'] = ele_cos_df['Capital costs']/ele_cos_df['Electricity generation']
ele_cos_df['Fixed costs'] = fix_cos_df.iloc[:,1:].sum(axis=1)/ele_cos_df['Electricity generation']
ele_cos_df['Variable costs'] = var_cos_df.iloc[:,1:].sum(axis=1)/ele_cos_df['Electricity generation']
ele_cos_df['Fuel distribution costs'] = dis_cos_df.iloc[:,1:].sum(axis=1)/ele_cos_df['Electricity generation']
ele_cos_df['Fuel costs'] = fue_cos_df.iloc[:,1:].sum(axis=1)/ele_cos_df['Electricity generation']

In [ ]:
ele_cos_df.drop('Electricity generation',axis=1,inplace=True)

In [ ]:
ele_cos_df.iplot(kind='bar',barmode='stack',x='y',title='Cost of electricity generation ($/MWh)')

In [ ]:
# Land use

regions = {'C':'Center',
           'N':'North',
           'S':'South'}

crops = {'CP1':'Coffee',
         'CP2':'Cotton',
         'CP3':'Peanuts',
         'CP4':'Maize',
         'CP5':'Rice',
         'CP6':'Soybean',
         'CP7':'Sugarcane',
         'CP8':'Tea'}

water_supply = {'I':'Irrigated',
                'R':'Rain-fed'}

input_level = {'L':'Low',
               'I':'Intermediate',
               'H':'High'}

In [ ]:
crops_total_df = all_params['TotalTechnologyAnnualActivity'][all_params['TotalTechnologyAnnualActivity'].t.str.startswith('LNDCP')].drop('r', axis=1)
crops_total_df = df_filter(crops_total_df,3,6,[])
df_plot(crops_total_df,'Land area (1000 sq.km.)','Area by crop')

In [ ]:
land_total_df = all_params['TotalTechnologyAnnualActivity'][all_params['TotalTechnologyAnnualActivity'].t.str.startswith('LND')].drop('r', axis=1)
land_total_df = df_filter(land_total_df,3,6,['AGR'])
land_total_df['Agriculture'] = 0
for each in ['Coffee','Cotton','Peanuts','Maize','Rice','Soybean','Sugarcane','Tea']:
    if each in land_total_df:
        land_total_df['Agriculture'] = land_total_df['Agriculture'] + land_total_df[each]
        land_total_df = land_total_df.drop(each,axis=1)
df_plot(land_total_df,'Land area (1000 sq.km.)','Area by land cover type')

In [ ]:
for each in regions.keys():
    crops_region_df = all_params['TotalTechnologyAnnualActivity'][all_params['TotalTechnologyAnnualActivity'].t.str.startswith('LNDCP')].drop('r', axis=1)
    crops_region_df = crops_region_df[crops_region_df.t.str[6:7] == each]
    crops_region_df = df_filter(crops_region_df,3,6,[])
    df_plot(crops_region_df,'Land area (1000 sq.km.)','Area by crop (' + regions[each] + ' region)')

In [ ]:
for each in regions.keys():
    land_region_df = all_params['TotalTechnologyAnnualActivity'][all_params['TotalTechnologyAnnualActivity'].t.str.startswith('LND')].drop('r', axis=1)
    land_region_df = land_region_df[land_region_df.t.str[6:7] == each]
    land_region_df = df_filter(land_region_df,3,6,['AGR'])
    land_region_df['Agriculture'] = 0
    for crop in ['Coffee','Cotton','Peanuts','Maize','Rice','Soybean','Sugarcane','Tea']:
        if crop in land_region_df:
            land_region_df['Agriculture'] = land_region_df['Agriculture'] + land_region_df[crop]
            land_region_df = land_region_df.drop(crop,axis=1)
    df_plot(land_region_df,'Land area (1000 sq.km.)','Area by land cover type (' + regions[each] + ' region)')

In [ ]:
for each in water_supply.keys():
    crops_ws_df = all_params['TotalTechnologyAnnualActivity'][all_params['TotalTechnologyAnnualActivity'].t.str.startswith('LNDCP')].drop('r', axis=1)
    crops_ws_df = crops_ws_df[crops_ws_df.t.str[8:9] == each]
    crops_ws_df = df_filter(crops_ws_df,3,6,[])
    df_plot(crops_ws_df,'Land area (1000 sq.km.)','Area by crop (' + water_supply[each] + ')')

In [ ]:
crops_prod_df = all_params['ProductionByTechnologyAnnual'][all_params['ProductionByTechnologyAnnual'].f.str.startswith('CRP')].drop('r', axis=1)
crops_prod_df['f'] = crops_prod_df['f'].str[3:6]
crops_prod_df['value'] = crops_prod_df['value'].astype('float64').div(10)

crops_prod_df = crops_prod_df.pivot_table(index='y', 
                                          columns='f',
                                          values='value',
                                          aggfunc='sum').reset_index().fillna(0)
crops_prod_df = crops_prod_df.reindex(sorted(crops_prod_df.columns), axis=1).set_index('y').reset_index().rename(columns=det_col)
crops_prod_df['y'] = years
df_plot(crops_prod_df,'Production (Million tonnes)','Crop production')

In [ ]:
crops_yield_df = crops_prod_df*10/crops_total_df
crops_yield_df['y'] = years

crops_yield_df.iplot(x='y',
                     mode='lines+markers', 
                     xTitle='Year',
                     yTitle='Yield (t/ha)',
                     size=10,
                     color=[color_dict[x] for x in crops_yield_df.columns if x != 'y'],
                     title='Yield (tonnes/hectare)')

In [ ]:
wat_dem_df = all_params['ProductionByTechnologyAnnual'][all_params['ProductionByTechnologyAnnual'].f.str[0:6].isin(['AGRWAT','PUBWAT','PWRWAT'])].drop('r', axis=1)
wat_dem_df['f'] = wat_dem_df['f'].str[0:3]
wat_dem_df['value'] = wat_dem_df['value'].astype('float64')
wat_dem_df = wat_dem_df.pivot_table(index='y', 
                                          columns='f',
                                          values='value',
                                          aggfunc='sum').reset_index().fillna(0)

In [ ]:
wat_bal_df = all_params['ProductionByTechnologyAnnual'][all_params['ProductionByTechnologyAnnual'].f.str.startswith('WTR')].drop('r', axis=1)
wat_bal_df['f'] = wat_bal_df['f'].str[3:6]
wat_bal_df['value'] = wat_bal_df['value'].astype('float64')
wat_bal_df = wat_bal_df.pivot_table(index='y', 
                                          columns='f',
                                          values='value',
                                          aggfunc='sum').reset_index().fillna(0)
wat_bal_df = wat_bal_df.reindex(sorted(wat_bal_df.columns), axis=1).set_index('y').reset_index().rename(columns=det_col)
wat_bal_df['Irrigation'] = wat_dem_df['AGR']
wat_bal_df['y'] = years
df_plot(wat_bal_df,'Billion m3', 'Water balance')